### Ques 1

In [10]:
"""Python implementation of Dijkstra’s algorithm extended to:

Work on directed graphs.

Compute all-pairs shortest paths.

Print all possible shortest paths if multiple exist.

From a single source, identify edge-disjoint shortest paths."""

'Python implementation of Dijkstra’s algorithm extended to:\n\nWork on directed graphs.\n\nCompute all-pairs shortest paths.\n\nPrint all possible shortest paths if multiple exist.\n\nFrom a single source, identify edge-disjoint shortest paths.'

In [2]:
import heapq
from collections import defaultdict, deque

class Graph:
    def __init__(self, n):
        self.n = n                      # number of vertices
        self.adj = defaultdict(list)    # adjacency list {u: [(v, w), ...]}

    def add_edge(self, u, v, w):
        # directed edge u -> v with weight w
        self.adj[u].append((v, w))

    # ---------------- Part 1: Single-Source Dijkstra ----------------
    def dijkstra_all_paths(self, src):
        """ Run Dijkstra from src, return distances and all shortest paths. """
        dist = [float('inf')] * (self.n + 1)
        dist[src] = 0

        # parents[v] = list of predecessors on shortest paths
        parents = [[] for _ in range(self.n + 1)]

        pq = [(0, src)]  # (distance, node)
        while pq:
            d, u = heapq.heappop(pq)
            if d > dist[u]:
                continue
            for v, w in self.adj[u]:
                if dist[v] > dist[u] + w:
                    dist[v] = dist[u] + w
                    parents[v] = [u]
                    heapq.heappush(pq, (dist[v], v))
                elif dist[v] == dist[u] + w:
                    # another shortest path found
                    parents[v].append(u)
        return dist, parents

    def get_all_paths(self, parents, src, dest):
        """ Backtrack using parents to generate all shortest paths. """
        paths = []
        def dfs(v, path):
            if v == src:
                paths.append(path[::-1])
                return
            for p in parents[v]:
                dfs(p, path + [p])
        dfs(dest, [dest])
        return paths

    # ---------------- Part 2: All-Pairs Shortest Paths ----------------
    def all_pairs_shortest_paths(self):
        all_distances = {}
        all_paths = {}
        for src in range(1, self.n + 1):
            dist, parents = self.dijkstra_all_paths(src)
            all_distances[src] = dist
            all_paths[src] = {}
            for dest in range(1, self.n + 1):
                if dist[dest] < float('inf'):
                    all_paths[src][dest] = self.get_all_paths(parents, src, dest)
        return all_distances, all_paths

    # ---------------- Part 3: Edge-Disjoint Shortest Paths (Single Source) ----------------
    def edge_disjoint_paths(self, src):
        dist, parents = self.dijkstra_all_paths(src)
        edge_disjoint = {}
        for dest in range(1, self.n + 1):
            if dest == src or dist[dest] == float('inf'):
                continue
            paths = self.get_all_paths(parents, src, dest)
            disjoint_pairs = []
            for i in range(len(paths)):
                for j in range(i+1, len(paths)):
                    if self.are_edge_disjoint(paths[i], paths[j]):
                        disjoint_pairs.append((paths[i], paths[j]))
            if disjoint_pairs:
                edge_disjoint[dest] = disjoint_pairs
        return edge_disjoint

    def are_edge_disjoint(self, path1, path2):
        edges1 = {(path1[i], path1[i+1]) for i in range(len(path1)-1)}
        edges2 = {(path2[i], path2[i+1]) for i in range(len(path2)-1)}
        return edges1.isdisjoint(edges2)




In [3]:
# ---------------- Sample Run ----------------
if __name__ == "__main__":
    n, m = 5, 6
    edges = [
        (1, 2, 2),
        (1, 3, 3),
        (2, 3, 1),
        (2, 4, 7),
        (3, 5, 3),
        (4, 5, 2)
    ]

    g = Graph(n)
    for u, v, w in edges:
        g.add_edge(u, v, w)   # directed graph

    # Part 2 & 3: All-pairs shortest paths
    distances, paths = g.all_pairs_shortest_paths()
    print("All-pairs shortest paths:")
    for src in range(1, n+1):
        for dest in range(1, n+1):
            if src != dest and dest in paths[src]:
                for p in paths[src][dest]:
                    print(f"Shortest path ({src} -> {dest}): {' -> '.join(map(str,p))}, length = {distances[src][dest]}")

    # Part 4: Edge-disjoint paths from source=1
    print("\nEdge-disjoint shortest paths from source 1:")
    disjoint = g.edge_disjoint_paths(1)
    for dest, pairs in disjoint.items():
        for p1, p2 in pairs:
            print(f"To {dest}: {p1} AND {p2}")

All-pairs shortest paths:
Shortest path (1 -> 2): 1 -> 2, length = 2
Shortest path (1 -> 3): 1 -> 3, length = 3
Shortest path (1 -> 3): 1 -> 2 -> 3, length = 3
Shortest path (1 -> 4): 1 -> 2 -> 4, length = 9
Shortest path (1 -> 5): 1 -> 3 -> 5, length = 6
Shortest path (1 -> 5): 1 -> 2 -> 3 -> 5, length = 6
Shortest path (2 -> 3): 2 -> 3, length = 1
Shortest path (2 -> 4): 2 -> 4, length = 7
Shortest path (2 -> 5): 2 -> 3 -> 5, length = 4
Shortest path (3 -> 5): 3 -> 5, length = 3
Shortest path (4 -> 5): 4 -> 5, length = 2

Edge-disjoint shortest paths from source 1:
To 3: [1, 3] AND [1, 2, 3]


### ques 2

In [9]:
"""Python implementation of Bidirectional Dijkstra’s algorithm with:

Forward Dijkstra from source s (Aragog).

Backward Dijkstra from target t (Mosag).

Proper termination condition using 
𝜇.

Path reconstruction."""

'Python implementation of Bidirectional Dijkstra’s algorithm with:\n\nForward Dijkstra from source s (Aragog).\n\nBackward Dijkstra from target t (Mosag).\n\nProper termination condition using \n𝜇.\n\nPath reconstruction.'

In [5]:
import heapq
from collections import defaultdict

def bidirectional_dijkstra(n, edges, s, t):
    # Build adjacency list (undirected graph)
    graph = defaultdict(list)
    for u, v, w in edges:
        graph[u].append((v, w))
        graph[v].append((u, w))

    # Forward search (from source)
    dist_f = [float('inf')] * (n + 1)
    dist_f[s] = 0
    pq_f = [(0, s)]
    visited_f = set()
    parent_f = {s: None}

    # Reverse search (from target)
    dist_r = [float('inf')] * (n + 1)
    dist_r[t] = 0
    pq_r = [(0, t)]
    visited_r = set()
    parent_r = {t: None}

    # Best path length so far
    mu = float('inf')
    meeting_node = -1

    while pq_f and pq_r:
        # Current best priorities
        topf = pq_f[0][0] if pq_f else float('inf')
        topr = pq_r[0][0] if pq_r else float('inf')

        # Stop if termination condition satisfied
        if topf + topr >= mu:
            break

        # Expand forward frontier
        if topf <= topr:
            d, u = heapq.heappop(pq_f)
            if u in visited_f:
                continue
            visited_f.add(u)

            for v, w in graph[u]:
                if dist_f[v] > dist_f[u] + w:
                    dist_f[v] = dist_f[u] + w
                    parent_f[v] = u
                    heapq.heappush(pq_f, (dist_f[v], v))
                # Check overlap condition
                if v in visited_r:
                    if dist_f[u] + w + dist_r[v] < mu:
                        mu = dist_f[u] + w + dist_r[v]
                        meeting_node = v

        # Expand reverse frontier
        else:
            d, u = heapq.heappop(pq_r)
            if u in visited_r:
                continue
            visited_r.add(u)

            for v, w in graph[u]:
                if dist_r[v] > dist_r[u] + w:
                    dist_r[v] = dist_r[u] + w
                    parent_r[v] = u
                    heapq.heappush(pq_r, (dist_r[v], v))
                # Check overlap condition
                if v in visited_f:
                    if dist_r[u] + w + dist_f[v] < mu:
                        mu = dist_r[u] + w + dist_f[v]
                        meeting_node = v

    # ---------------- Reconstruct Path ----------------
    if mu == float('inf'):
        return None, []  # no path exists

    # Build path s -> meeting_node
    path_f = []
    node = meeting_node
    while node is not None:
        path_f.append(node)
        node = parent_f.get(node)
    path_f.reverse()

    # Build path meeting_node -> t
    path_r = []
    node = parent_r.get(meeting_node)
    while node is not None:
        path_r.append(node)
        node = parent_r.get(node)

    path = path_f + path_r
    return mu, path


In [6]:
# ---------------- Sample Run ----------------
if __name__ == "__main__":
    n, m = 6, 7
    edges = [
        (1, 2, 2),
        (1, 3, 4),
        (2, 4, 7),
        (3, 4, 1),
        (3, 5, 3),
        (4, 6, 1),
        (5, 6, 5)
    ]
    s, t = 1, 6

    dist, path = bidirectional_dijkstra(n, edges, s, t)
    print(f"Shortest distance from {s} to {t}: {dist}")
    print("Path:", " -> ".join(map(str, path)))

Shortest distance from 1 to 6: 6
Path: 1 -> 3 -> 4 -> 6


### ques 3

In [7]:
"""We’ll use Breadth-First Search (BFS) with an extra dimension in the state to track whether Andy has already broken a wall.

Key idea:

Each BFS state = (x, y, wall_break_used)

wall_break_used = 0 → no wall broken yet.

wall_break_used = 1 → already broke one wall.

From each cell:

If it’s 0 (corridor), move normally.

If it’s 1 (wall) and wall_break_used == 0, Andy can break it → move with wall_break_used = 1.

Track visited states as visited[x][y][wall_break_used].

This ensures we explore both possibilities (before and after breaking one wall)."""

'We’ll use Breadth-First Search (BFS) with an extra dimension in the state to track whether Andy has already broken a wall.\n\nKey idea:\n\nEach BFS state = (x, y, wall_break_used)\n\nwall_break_used = 0 → no wall broken yet.\n\nwall_break_used = 1 → already broke one wall.\n\nFrom each cell:\n\nIf it’s 0 (corridor), move normally.\n\nIf it’s 1 (wall) and wall_break_used == 0, Andy can break it → move with wall_break_used = 1.\n\nTrack visited states as visited[x][y][wall_break_used].\n\nThis ensures we explore both possibilities (before and after breaking one wall).'

In [11]:
"""implemented BFS with an extra state to track whether Andy has already broken one wall.

If a wall is encountered and broken == 0, Andy can break it.

Otherwise, he can only move through corridors.

If no escape is possible even after breaking a wall, it returns -1."""

'implemented BFS with an extra state to track whether Andy has already broken one wall.\n\nIf a wall is encountered and broken == 0, Andy can break it.\n\nOtherwise, he can only move through corridors.\n\nIf no escape is possible even after breaking a wall, it returns -1.'

In [12]:
from collections import deque

def shortest_escape_with_one_wall(grid):
    h, w = len(grid), len(grid[0])

    # visited[x][y][k] → visited at (x,y) with k walls broken
    visited = [[[False]*2 for _ in range(w)] for _ in range(h)]

    # BFS queue: (x, y, steps, wall_break_used)
    q = deque([(0, 0, 1, 0)])  # start with step=1 at (0,0)
    visited[0][0][0] = True

    # directions: up, down, left, right
    directions = [(1,0), (-1,0), (0,1), (0,-1)]

    while q:
        x, y, steps, broken = q.popleft()

        # reached goal
        if x == h-1 and y == w-1:
            return steps

        for dx, dy in directions:
            nx, ny = x+dx, y+dy
            if 0 <= nx < h and 0 <= ny < w:
                # corridor
                if grid[nx][ny] == 0 and not visited[nx][ny][broken]:
                    visited[nx][ny][broken] = True
                    q.append((nx, ny, steps+1, broken))

                # wall, but we can break it
                if grid[nx][ny] == 1 and broken == 0 and not visited[nx][ny][1]:
                    visited[nx][ny][1] = True
                    q.append((nx, ny, steps+1, 1))

    return -1  # no escape possible





In [13]:
# ---------------- Sample Runs ----------------
if __name__ == "__main__":
    # Example 1 (from problem)
    grid1 = [
        [0,1,0,0],
        [0,1,0,1],
        [0,0,0,1],
        [1,1,0,0]
    ]
    print(shortest_escape_with_one_wall(grid1))  # Expected: 6

    # Example 2 (no walls to break)
    grid2 = [
        [0,0,0],
        [0,0,0],
        [0,0,0]
    ]
    print(shortest_escape_with_one_wall(grid2))  # Expected: 5 (straight path)

7
5
